<a href="https://colab.research.google.com/github/takumi-maker/bert/blob/main/momentum_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import math
import statistics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression

In [2]:
df_2 = pd.read_csv("債券複利データセット3.csv")
df_2 = df_2.drop(df_2.columns[0], axis=1)
columns_len = len(df_2.columns)
length = len(df_2)
df_2
y_df2 = df_2["GJGC10"]
X_df2 = df_2.drop("GJGC10", axis=1)
y_df2

,GJGC10
0,0.030
1,0.025
2,0.030
3,0.030
4,0.045
...,...
974,1.114
975,1.114
976,1.176
977,1.172


In [3]:
newest_list =[[1.200]]
new_df = pd.DataFrame(newest_list,columns=["GJGC10"])
new_df

,GJGC10
0,1.2


In [4]:
y_df2 = pd.concat([y_df2, new_df], axis=0)
y_df2

,GJGC10
0,0.030
1,0.025
2,0.030
3,0.030
4,0.045
...,...
975,1.114
976,1.176
977,1.172
978,1.195


In [ ]:
df_array = y_df2.to_numpy()
df_array

In [6]:
#モメンタム順張り
from sklearn.linear_model import LinearRegression
import statsmodels
import statsmodels.api
import matplotlib.pyplot as plt
from sklearn import preprocessing
import cvxpy as cp

magic_num = 30
estimate_num = 150

df_array = y_df2.to_numpy()

for i in range(1,8):
  total_num = 100*i
  if i != 1:
    break
  print(total_num)


  total_list = []
  y_list = []
  result_list = []
  predict_flag2 = 0
  for j in range(magic_num-2,len(df_array)):
    X_return_list = []
    for k in range(0,magic_num):
      X_return_list.append((df_array[j-k]-df_array[j-k-1])*1)
    try:
      y_return = (df_array[j+1]-df_array[j])*1
    except:
      y_return = 0
    y_list.append(y_return)
    X_list2 = list(reversed(X_return_list))
    total_list.append(X_list2)

  print(len(total_list))

  for j in range(len(total_list)):
    if j <= len(total_list)-1-total_num:
      continue
    if j != len(total_list)-1:
      continue
    #print(j)
    X_array = np.squeeze(np.array(total_list[j]))
    y_array = y_list[j]
    estimate_list = []
    estimate_y_list = []
    for k in range(1,estimate_num):
      estimate_list.append(total_list[j-k])
      estimate_y_list.append(y_list[j-k])

    estimate_list = np.array(estimate_list)
    estimate_list = np.squeeze(estimate_list)
    estimate_y_list = np.array(estimate_y_list)
    #print(estimate_list)
    #print(estimate_y_list)


    #alpha = np.array([0,0.1,0.2,0.3,0.4,0.5])
    #print(estimate_list.shape)
    #print(estimate_y_list.shape)
    k= estimate_list.shape[1]
    beta = cp.Variable(k,nonneg=True)
    constraints =[beta[i] <= beta[i+1] for i in range(k-1)]
    #error = cp.norm2(estimate_y_list - estimate_list @ beta)
    objective =  cp.sum_squares(estimate_y_list - estimate_list @ beta)
    #objective = cp.Minimize(error)


    #prob = cp.Problem(objective, constraints)
    prob = cp.Problem(cp.Minimize(objective), constraints)
    prob.solve()
    #print("status:", prob.status)
    #print("optimal value", prob.value)
    #print("optimal var", beta.value)

    predict = np.dot(X_array,beta.value)
    if j == len(total_list)-1:
      print(predict)
    #print(predict)
    #print(y_array)

    if predict >= 0:
      predict_flag = 1
    else:
      predict_flag = -1

    if predict_flag*predict_flag2 >=0:
      result_r = y_array*predict_flag
      result_list.append(result_r)
      predict_flag2 = predict_flag
    else:
      result_r = y_array*predict_flag-0.005
      result_list.append(result_r)
      predict_flag2 = predict_flag

  R_sum = sum(result_list)
  SR = np.mean(np.array(result_list))/np.std(np.array(result_list))
  print("SharpRatio:",SR)
  print(R_sum)



100
952


/usr/local/lib/python3.11/dist-packages/cvxpy/reductions/solvers/solving_chain_utils.py:30: UserWarning: The problem includes expressions that don't support CPP backend. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(


0.000359368151854372
SharpRatio: nan
0


<ipython-input-6-4c7f4aa1cb44>:99: RuntimeWarning: invalid value encountered in scalar divide
  SR = np.mean(np.array(result_list))/np.std(np.array(result_list))


In [11]:
#predict2
from sklearn.linear_model import LinearRegression
import statsmodels
import statsmodels.api
import matplotlib.pyplot as plt
from sklearn import preprocessing
import cvxpy as cp

magic_num = 30
estimate_num = 150

df_array = y_df2.to_numpy()

for i in range(1,8):
  total_num = 100*i
  print(total_num)

  total_list = []
  y_list = []
  result_list = []
  predict_flag2 = 0
  for j in range(magic_num-2,len(df_array)+1):
    X_return_list = []
    for k in range(1,magic_num):
      X_return_list.append((df_array[j-k]-df_array[j-k-1])*1)
    if j == len(df_array):
      y_return = 0
    else:
      y_return = (df_array[j]-df_array[j-1])*1
    y_list.append(y_return)
    X_list2 = list(reversed(X_return_list))
    total_list.append(X_list2)
  #print(len(total_list))

  for j in range(len(total_list)):
    if j <= len(total_list)-1-total_num:
      continue
    if j != len(total_list)-1:
      continue
    #print(j)
    X_array = np.squeeze(np.array(total_list[j]))
    y_array = y_list[j]
    estimate_list = []
    estimate_y_list = []
    for k in range(1,estimate_num):
      estimate_list.append(total_list[j-k])
      estimate_y_list.append(y_list[j-k])

    estimate_list = np.array(estimate_list)
    estimate_list = np.squeeze(estimate_list)
    estimate_y_list = np.array(estimate_y_list)
    #print(estimate_list)
    #print(estimate_y_list)


    #alpha = np.array([0,0.1,0.2,0.3,0.4,0.5])
    #print(estimate_list.shape)
    #print(estimate_y_list.shape)
    k= estimate_list.shape[1]
    beta = cp.Variable(k,nonneg=True)
    constraints =[beta[i] <= beta[i+1] for i in range(k-1)]
    #error = cp.norm2(estimate_y_list - estimate_list @ beta)
    objective =  cp.sum_squares(estimate_y_list - estimate_list @ beta)
    #objective = cp.Minimize(error)


    #prob = cp.Problem(objective, constraints)
    prob = cp.Problem(cp.Minimize(objective), constraints)
    prob.solve()
    #print("status:", prob.status)
    #print("optimal value", prob.value)
    #print("optimal var", beta.value)

    predict = np.dot(X_array,beta.value)
    if j == len(total_list)-1:
      print(predict)
    #print(predict)
    #print(y_array)

    if predict >= 0:
      predict_flag = 1
    else:
      predict_flag = -1

    if predict_flag*predict_flag2 >=0:
      result_r = y_array*predict_flag
      result_list.append(result_r)
      predict_flag2 = predict_flag
    else:
      result_r = y_array*predict_flag-0.005
      result_list.append(result_r)
      predict_flag2 = predict_flag

  R_sum = sum(result_list)
  SR = np.mean(np.array(result_list))/np.std(np.array(result_list))
  print("SharpRatio:",SR)
  print(R_sum)

100
0.00031736341322384656
SharpRatio: nan
0
200


<ipython-input-11-5c0ea8f5bcf3>:95: RuntimeWarning: invalid value encountered in scalar divide
  SR = np.mean(np.array(result_list))/np.std(np.array(result_list))


Solver interrupted


SolverError: Solver 'OSQP' failed. Try another solver, or solve with verbose=True for more information.

In [ ]:
#モメンタム逆張り
from sklearn.linear_model import LinearRegression
import statsmodels
import statsmodels.api
import matplotlib.pyplot as plt
from sklearn import preprocessing
import cvxpy as cp

magic_num = 30
estimate_num = 150

df_array = y_df2.to_numpy()

for i in range(1,8):
  total_num = 100*i
  print(total_num)

  total_list = []
  y_list = []
  result_list = []
  predict_flag2 = 0
  for j in range(magic_num-2,len(df_array)):
    X_return_list = []
    for k in range(1,magic_num):
      X_return_list.append((df_array[j-k]-df_array[j-k-1])*1)
    y_return = (df_array[j]-df_array[j-1])*1
    y_list.append(y_return)
    X_list2 = list(reversed(X_return_list))
    total_list.append(X_list2)
  #print(len(total_list))

  for j in range(len(total_list)):
    if j <= len(total_list)-1-total_num:
      continue
    #print(j)
    X_array = np.squeeze(np.array(total_list[j]))
    y_array = y_list[j]
    estimate_list = []
    estimate_y_list = []
    for k in range(1,estimate_num):
      estimate_list.append(total_list[j-k])
      estimate_y_list.append(y_list[j-k])

    estimate_list = np.array(estimate_list)
    estimate_list = np.squeeze(estimate_list)
    estimate_y_list = np.array(estimate_y_list)
    #print(estimate_list)
    #print(estimate_y_list)


    #alpha = np.array([0,0.1,0.2,0.3,0.4,0.5])
    #print(estimate_list.shape)
    #print(estimate_y_list.shape)
    k= estimate_list.shape[1]
    beta = cp.Variable(k,nonneg=True)
    constraints =[beta[i] <= beta[i+1] for i in range(k-1)]
    #error = cp.norm2(estimate_y_list - estimate_list @ beta)
    objective =  cp.sum_squares(estimate_y_list - estimate_list @ beta)
    #objective = cp.Minimize(error)


    #prob = cp.Problem(objective, constraints)
    prob = cp.Problem(cp.Minimize(objective), constraints)
    prob.solve()
    #print("status:", prob.status)
    #print("optimal value", prob.value)
    #print("optimal var", beta.value)

    predict = np.dot(X_array,beta.value)
    #print(predict)
    #print(y_array)

    if predict >= 0.005:
      predict_flag = -1
    elif predict <-0.005:
      predict_flag = 1
    else:
      predict_flag = predict_flag2

    if predict_flag*predict_flag2 >=0:
      result_r = y_array*predict_flag
      result_list.append(result_r)
      predict_flag2 = predict_flag
    else:
      result_r = y_array*predict_flag-0.005
      result_list.append(result_r)
      predict_flag2 = predict_flag

  R_sum = sum(result_list)
  SR = np.mean(np.array(result_list))/np.std(np.array(result_list))
  print("SharpRatio:",SR)
  print(R_sum)

100
Solver interrupted


SolverError: Solver 'OSQP' failed. Try another solver, or solve with verbose=True for more information.

In [ ]:
!pip install cvxpy